In [9]:
import cv2
import numpy as np
import os
import pickle

In [13]:
with open('processed_images.pickle', 'rb') as f:
    processed_images = pickle.load(f)
processed_images, len(processed_images)

(['148.jpeg',
  '109.jpeg',
  '125.jpeg',
  '099.jpeg',
  '133.jpeg',
  '113.jpeg',
  '144.jpeg',
  '152.jpeg',
  '105.jpeg',
  '083.jpeg',
  '129.jpeg',
  '095.jpeg',
  '094.jpeg',
  '082.jpeg',
  '128.jpeg',
  '104.jpeg',
  '153.jpeg',
  '145.jpeg',
  '112.jpeg',
  '098.jpeg',
  '132.jpeg',
  '124.jpeg',
  '108.jpeg',
  '149.jpeg',
  '115.jpeg',
  '142.jpeg',
  '154.jpeg',
  '103.jpeg',
  '085.jpeg',
  '139.jpeg',
  '093.jpeg',
  '119.jpeg',
  '158.jpeg',
  '123.jpeg',
  '089.jpeg',
  '135.jpeg',
  '134.jpeg',
  '122.jpeg',
  '088.jpeg',
  '159.jpeg',
  '118.jpeg',
  '138.jpeg',
  '092.jpeg',
  '084.jpeg',
  '102.jpeg',
  '155.jpeg',
  '143.jpeg',
  '114.jpeg',
  '091.jpeg',
  '087.jpeg',
  '101.jpeg',
  '156.jpeg',
  '140.jpeg',
  '117.jpeg',
  '160.jpeg',
  '137.jpeg',
  '121.jpeg',
  '120.jpeg',
  '136.jpeg',
  '116.jpeg',
  '141.jpeg',
  '157.jpeg',
  '100.jpeg',
  '086.jpeg',
  '090.jpeg',
  '131.jpeg',
  '127.jpeg',
  '097.jpeg',
  '081.jpeg',
  '107.jpeg',
  '150.jpeg',
  '146

In [12]:
with open('window_labels.pickle', 'rb') as f:
    window_labels = pickle.load(f)
window_labels

        

{'220_730_0_148.jpeg': 0,
 '220_740_0_148.jpeg': 0,
 '220_750_0_148.jpeg': 0,
 '220_760_0_148.jpeg': 0,
 '220_770_0_148.jpeg': 0,
 '220_780_0_148.jpeg': 0,
 '220_790_0_148.jpeg': 0,
 '220_800_0_148.jpeg': 0,
 '220_810_0_148.jpeg': 0,
 '220_820_0_148.jpeg': 0,
 '220_830_0_148.jpeg': 0,
 '220_840_0_148.jpeg': 0,
 '220_850_0_148.jpeg': 0,
 '220_860_0_148.jpeg': 0,
 '220_870_0_148.jpeg': 0,
 '220_880_0_148.jpeg': 0,
 '220_890_0_148.jpeg': 0,
 '220_900_0_148.jpeg': 0,
 '220_910_0_148.jpeg': 0,
 '220_920_0_148.jpeg': 0,
 '220_930_0_148.jpeg': 0,
 '230_690_0_148.jpeg': 0,
 '230_700_0_148.jpeg': 0,
 '230_710_0_148.jpeg': 0,
 '230_720_0_148.jpeg': 0,
 '230_730_0_148.jpeg': 0,
 '230_740_0_148.jpeg': 0,
 '230_750_0_148.jpeg': 0,
 '230_760_0_148.jpeg': 0,
 '230_770_0_148.jpeg': 0,
 '230_780_0_148.jpeg': 0,
 '230_790_0_148.jpeg': 0,
 '230_800_0_148.jpeg': 0,
 '230_810_0_148.jpeg': 0,
 '230_820_0_148.jpeg': 0,
 '230_830_0_148.jpeg': 0,
 '230_840_0_148.jpeg': 0,
 '230_850_0_148.jpeg': 0,
 '230_860_0_

In [29]:
def load_images_from_folder(folder):
    image_tuples = []
    for filename in os.listdir(folder):
        if (filename not in processed_images):
            img = cv2.imread(os.path.join(folder,filename))
            if (img is not None):
                image_tuples.append((filename, img))
    return image_tuples

In [30]:
# MinMax Scaler
def minmaxScaler(image):
    minPixel = np.amin(image)
    maxPixel = np.amax(image)
    image = (image - minPixel) / (maxPixel - minPixel)
    return image

In [31]:
def threshold_image(image, threshold=0.1):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = minmaxScaler(gray)
    for i in range(len(gray)):
        for j in range(len(gray[i])):
            gray[i][j] = int((gray[i][j] > threshold) and not (image[i][j][0] < 255 * (1 - threshold) and image[i][j][1] < 255 * (1 - threshold) and image[i][j][2] > 255 * threshold)) * 255
    return gray

In [32]:
def generate_windows(image, filename, window_size, interior_size, increment, folder):
    gray_img = threshold_image(image)
    x = 0
    while (x < image.shape[0] - window_size):
        row_has_intersection = False
        y = 0
        while (y < image.shape[1] - window_size):
            window = image[x : x + window_size, y : y + window_size, :]
            has_intersection = False
            
            gray = gray_img[x : x + window_size, y : y + window_size]
            sum_of_grayscale_pixels = np.sum(gray) / 255
            
            # if the image is not completely white, we can add it to the dataset
            if (sum_of_grayscale_pixels < window_size * window_size) and sum_of_grayscale_pixels >= window_size * window_size/10:
                
                # check center interior square for any red pixels
                for x_interior in range (int(window_size/2 - interior_size/2), int(window_size/2 + interior_size/2)):
                    for y_interior in range (int(window_size/2 - interior_size/2), int(window_size/2 + interior_size/2)): 
                        # check if pixel is red
                        if (not has_intersection and 
                            window[x_interior][y_interior][0] < 100 and 
                            window[x_interior][y_interior][1] < 100 and
                            window[x_interior][y_interior][2] > 200):
                            row_has_intersection = True
                            has_intersection = True

                window_id = str(x) + "_" + str(y) + "_" + str(int(has_intersection)) + "_" + filename
                window_labels[window_id] = int(has_intersection)
                cv2.imwrite(os.path.join(folder, window_id), gray)
                
            if (has_intersection):
                y += int(increment / 5)
            else:
                y += increment
        if (row_has_intersection):
            x += int(increment / 5)
        else:
            x += increment

    processed_images.append(filename)

In [33]:
image_tuples = load_images_from_folder("images")
for image_tuple in image_tuples:
    filename = image_tuple[0]
    image = image_tuple[1]
    generate_windows(image=image, filename=filename, window_size=25, interior_size=5, increment=10, folder="dataset")

In [34]:
with open('processed_images.pickle', 'wb') as f:
    pickle.dump(processed_images, f)

In [35]:
with open('window_labels.pickle', 'wb') as f:
    pickle.dump(window_labels, f)

In [26]:
processed_images = []
window_labels = {}

In [27]:
window_labels

{}

In [28]:
print(len(processed_images))
processed_images

0


[]

In [40]:
import shutil
def new_threshold(gray, threshold=25):
    for i in range(len(gray)):
        for j in range(len(gray[i])):
            gray[i][j] = int(gray[i][j] > threshold) * 255
    return gray
directory = "dataset"
# new_directory = "test"
counter = 0
new_counter = 0
for filename in os.listdir(directory):
#     print(filename, directory, os.path.join(directory, filename))
#     if counter < 100:
#         print(filename)
#     if counter > 30000:
#         break
    img = cv2.imread(os.path.join(directory,filename), 0)
    img = new_threshold(img)
#     img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    counter += 1
    if np.sum(img)/255 < 60:
        new_counter += 1
#         shutil.move(os.path.join(directory, filename), os.path.join(new_directory, filename))

print(counter, new_counter)
        


129405 0
